In [ ]:
import os
import time
import poppytools

from pypot.robot import from_json

config_path = os.path.join(os.path.dirname(poppytools.__file__), 'configuration', 'poppy_config.json')

poppy = from_json(config_path)
poppy.start_sync()

In [ ]:
import json

def save_pos(filename, pos):
    with open(filename, 'w') as f:
        json.dump(pos, f, indent=2)
        
def load_pos(filename):
    with open(filename) as f:
        return json.load(f)
    
def get_pos(motors):
    return {m.name: m.present_position for m in motors}

def set_pos(pos):
    for n, p in pos.items():
        getattr(poppy, n).goal_position = p

In [ ]:
from pyo import *

s = Server().boot()
s.start()

In [ ]:
bob_sf = [
    SfPlayer('sounds/kicks/kick08_good.wav', mul=0.8).out(),
    SfPlayer('sounds/snare05.wav', mul=0.8).out(),
    SfPlayer('sounds/chorus03.wav', mul=0.8).out(),
    SfPlayer('sounds/chorus01.ogg', mul=0.8).out(),
]

class Sampler(object):
    def play(self, id):
        bob_sf[id].play().out()
        
sampler = Sampler()

In [ ]:
s.recstart(filename='6.wav')


In [ ]:
from pypot.primitive import LoopPrimitive

class Blah(LoopPrimitive):
    def setup(self):
        self.play = False
        
    def update(self):
        if poppy.r_shoulder_y.present_position < -50 and not self.play:
            self.play = True
            
            x = poppy.r_shoulder_x.present_position + 25
            i = 2 if x > 0 else 3
            
            sampler.play(i)
            
        if poppy.r_shoulder_y.present_position > -40 and self.play:
            self.play = False
            
class Spoueh(LoopPrimitive):
    def setup(self):
        self.play = False
        
    def update(self):
        if poppy.l_ankle_y.present_position > -15 and not self.play:
            self.play = True
            sampler.play(0)
            
        if poppy.l_ankle_y.present_position < -20 and self.play:
            self.play = False
            
blah = Blah(poppy, 50)
blah.start()

spoueh = Spoueh(poppy, 50)
spoueh.start()

In [ ]:
poppy.r_arm_z


In [ ]:
live.start()
sf.play()


In [ ]:
for m in [poppy.l_ankle_y,]:
    m.compliant = True

In [ ]:
for m in [poppy.l_ankle_y, ]:
    m.compliant = False
    
play_measure(move=True)

In [ ]:
sf.stop()
live.stop()

In [ ]:
s.recstop()

In [ ]:
SfPlayer('2eme_prise.wav', mul=0.8).out()

In [ ]:
from pypot.vrep import from_vrep
import time
import numpy as np

# Connexion au simulateur V-REP
poppy = from_vrep('poppy-humanoid')

# Définition des moteurs clés
motors = {
    "hip_left": poppy.l_hip_y,
    "hip_right": poppy.r_hip_y,
    "knee_left": poppy.l_knee_y,
    "knee_right": poppy.r_knee_y,
    "ankle_left": poppy.l_ankle_y,
    "ankle_right": poppy.r_ankle_y
}

# Définition de la trajectoire de marche avec interpolation
def walking_sequence(step_duration=1, step_size=15, steps=5):
    for _ in range(steps):
        # 1. Avancer la jambe droite
        motors["hip_right"].goal_position = step_size
        motors["knee_right"].goal_position = -step_size
        motors["ankle_right"].goal_position = step_size / 2
        
        time.sleep(step_duration / 2)

        # 2. Reposer la jambe droite et lever la gauche
        motors["hip_right"].goal_position = 0
        motors["knee_right"].goal_position = 0
        motors["ankle_right"].goal_position = 0
        
        motors["hip_left"].goal_position = step_size
        motors["knee_left"].goal_position = -step_size
        motors["ankle_left"].goal_position = step_size / 2

        time.sleep(step_duration / 2)

        # 3. Revenir en position initiale
        motors["hip_left"].goal_position = 0
        motors["knee_left"].goal_position = 0
        motors["ankle_left"].goal_position = 0

        time.sleep(step_duration / 2)

# Exécution de la marche
print("Démarrage de la marche...")
walking_sequence()
print("Marche terminée.")


In [ ]:
import time
import numpy as np
from pypot.robots import from_json
from pypot.primitive import LoopPrimitive
from poppy_humanoid import PoppyHumanoid

class WalkingController(LoopPrimitive):
    def __init__(self, robot):
        LoopPrimitive.__init__(self, robot, 50)  # 50 Hz
        self.phase = 0.0
        self.params = {
            'step_height': 0.15,
            'step_length': 0.2,
            'cycle_time': 2.0,
            'hip_amplitude': 25,
            'knee_amplitude': 20
        }

    def update(self):
        dt = 1.0 / self.frequency
        self.phase += dt / self.params['cycle_time']

        # Calcul des angles avec modèle cinématique
        angles = self.calculate_walk_angles()
        
        # Application des positions aux moteurs
        for motor, angle in angles.items():
            getattr(self.robot, motor).goal_position = angle

    def calculate_walk_angles(self):
        phase = self.phase % 1.0
        
        # Pattern de marche cyclique
        l_hip_y = self.params['hip_amplitude'] * np.sin(2 * np.pi * phase)
        r_hip_y = -self.params['hip_amplitude'] * np.sin(2 * np.pi * phase)
        
        l_knee_y = 40 + self.params['knee_amplitude'] * np.sin(2 * np.pi * phase)
        r_knee_y = 40 - self.params['knee_amplitude'] * np.sin(2 * np.pi * phase)

        return {
            'l_hip_y': l_hip_y,
            'r_hip_y': r_hip_y,
            'l_knee_y': l_knee_y,
            'r_knee_y': r_knee_y,
            'l_ankle_y': -l_knee_y/2,
            'r_ankle_y': -r_knee_y/2
        }

# Initialisation du robot dans V-REP
with PoppyHumanoid(simulator='vrep') as robot:
    # Configuration initiale
    robot.compliant = False
    robot.start_simulation()
    
    # Création et démarrage du contrôleur
    walker = WalkingController(robot)
    walker.start()
    
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        walker.stop()
        robot.stop_simulation()

In [ ]:
from poppy_humanoid import PoppyHumanoid
import time

# Configuration de base
robot= PoppyHumanoid(simulator='vrep')
robot.start_simulation()
print(robot.motors)
    # Exemple de mouvement
robot.l_knee_y.goto_position( 90, 2.0, wait=True)
robot.r_knee_y.goto_position( 90, 2.0, wait=True)
time.sleep(5)
robot.stop_simulation()
robot.close()

In [ ]:
from poppy_humanoid import PoppyHumanoid
import time
import numpy as np

def twerking_animation(amplitude=40, speed=2):
    """Génère les angles de mouvement pour le twerk"""
    t = time.time()
    angle = amplitude * np.sin(speed * 4 * np.pi * t)
    
    return {
        'l_hip_y': -angle,
        'r_hip_y': -angle,
        'l_knee_y': 90 + 0.5*angle,
        'r_knee_y': 90 + 0.5*angle,
        'l_ankle_y': -20 + 0.2*angle,
        'r_ankle_y': -20 + 0.2*angle
    }

robot= PoppyHumanoid(simulator='vrep')
robot.start_simulation()
robot.compliant = False  # Désactive la compliance

print("Twerking time! Ctrl+C pour arrêter")

try:
    while True:
        # Calcule les nouvelles positions
        angles = twerking_animation(
            amplitude=45,  # Plus grand = mouvement plus large
            speed=2       # Plus grand = plus rapide
        )
        
        # Applique les positions aux moteurs
        for motor, angle in angles.items():
            getattr(robot, motor).goal_position = angle
            
        time.sleep(0.05)  # Contrôle la fluidité
except KeyboardInterrupt:
    robot.stop_simulation()
    print("\nArret du twerk!")
    robot.close()

In [ ]:
from poppy_humanoid import PoppyHumanoid
import time
import numpy as np
robot= PoppyHumanoid(simulator='vrep')
robot.start_simulation()
robot.abs_y.goto_position( 90, duration=2.0)
robot.bust_y.goto_position(90, duration=2)
robot.l_knee_y.goto_position( 90, 2.0)
robot.r_knee_y.goto_position( 90, 2.0)
robot.r_shoulder_y.goto_position(-60,2)
robot.l_shoulder_y.goto_position(-60,2)
robot.close()